In [1]:
import seaborn as sns
import pandas as pd
import befaas.logentry as le
from befaas.fileutil import log_entry_iterator


sns.set()

filepath = "../dumps/"
logdumps = ["dump_webservice_google.json","dump_webservice_aws.json","dump_webservice_azure.json"]
#logdumps = ["dump_webservice_azure.json"]
oufile = "../img/functionRuntimesEcomm.pkl"
print("Everything is set up.")


Everything is set up.


In [2]:
plotdata = []

for dump in logdumps:
    print(f"Include dump {dump} ...")
    # Count entries to get some progress feedback
    entries = 0
    contexts = {}
    otherUserAgentContextList = []

    for entry in log_entry_iterator(filepath + dump):
        entries = entries + 1
        if (entries % 10000 == 0):
            print(f"(1/2)  processed {entries} entries")

        logentry = le.cast_log_type(entry)
        if isinstance(logentry, le.RequestLog):
            userAgent = logentry.event['request']['headers']['user-agent']
            if not (str(userAgent).startswith("node-fetch") or str(userAgent).startswith("Artillery")):
                print(f"Other user agent, will filter: {userAgent}")
                otherUserAgentContextList.append(logentry.context_id)

    entries = 0
    for entry in log_entry_iterator(filepath + dump):
        entries = entries + 1
        if (entries % 10000 == 0):
            print(f"(2/2) processed {entries} entries")

        logentry = le.cast_log_type(entry)
        if isinstance(logentry, le.PerfLog):
            if logentry.type == "measure":
                if logentry.function == "frontend":
                    if not logentry.perf_type[1] == "rpcOut":

                        id = logentry.context_id

                        # Double-Check Context
                        context_ok = True

                        if id in otherUserAgentContextList:
                            context_ok = False
                            print(f"context filter applied to context {id}")

                        if context_ok:
                            # Create context if there isn't one
                            if not (id in contexts):
                                contexts[id] = []

                            # Check if there is already an entry for this context
                            if len(contexts[id]) > 0 :
                                context_ok = False
                                for checkEntry in contexts[id]:
                                    if str(checkEntry) != str(entry):
                                        # No duplicate -> problem
                                        print(f"ERROR: {id}: {len(contexts[id])}")
                                        tmp = le.cast_log_type(checkEntry)
                                        print(f"entry: {tmp.perf_type}#{tmp.type}#{tmp.perf_type_data}#{tmp.function}")
                                        print(f"{tmp}")
                                        print(f"NEW entry:")
                                        print(f"entry: {logentry.perf_type}#{logentry.type}#{logentry.perf_type_data}#{logentry.function}")
                                        print(f"{logentry}")

                            #Parse and add data entry if context check succeeded
                            if context_ok :
                                contexts[id].append(entry)

                                function = logentry.perf_type_data
                                duration = logentry.perf["duration"]
                                plattform = logentry.platform
                                #print(f"{id}#{function}#{duration}#{plattform}")
                                if (duration > 0):
                                    row = {}
                                    row["id"] = id
                                    row["function"] = function
                                    row["duration"] = duration
                                    row["plattform"] = plattform
                                    plotdata.append(row)
                                else:
                                    print(f"Error: {id}#{function}#{duration}#{plattform}")

Include dump dump_webservice_google.json ...
Iterate over ../dumps/dump_webservice_google.json ...
(1/2)  processed 10000 entries
(1/2)  processed 20000 entries
(1/2)  processed 30000 entries
(1/2)  processed 40000 entries
(1/2)  processed 50000 entries
(1/2)  processed 60000 entries
(1/2)  processed 70000 entries
(1/2)  processed 80000 entries
(1/2)  processed 90000 entries
(1/2)  processed 100000 entries
(1/2)  processed 110000 entries
(1/2)  processed 120000 entries
(1/2)  processed 130000 entries
(1/2)  processed 140000 entries
(1/2)  processed 150000 entries
(1/2)  processed 160000 entries
(1/2)  processed 170000 entries
(1/2)  processed 180000 entries
(1/2)  processed 190000 entries
(1/2)  processed 200000 entries
(1/2)  processed 210000 entries
(1/2)  processed 220000 entries
(1/2)  processed 230000 entries
(1/2)  processed 240000 entries
(1/2)  processed 250000 entries
(1/2)  processed 260000 entries
(1/2)  processed 270000 entries
(1/2)  processed 280000 entries
(1/2)  process

In [3]:
df_calls = pd.DataFrame(plotdata)
df_calls.set_index("id", drop=True, inplace=True)

df_calls.head()

,function,duration,plattform
id,,,
alat1zgg,/checkout,430.411724,google
b6thrhrv,/cart,162.080742,google
05zs9a2w,/product/:productId,78.121755,google
96avaqpj,/product/:productId,112.281232,google
8fcwchf4,/addCartItem,69.495420,google


In [4]:
df_calls.to_pickle(oufile)